#pip installations

In [1]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 6.5 MB 32.6 MB/s 
     |████████████████████████████████| 596 kB 42.3 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 895 kB 40.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 5.3 MB/s 
     |████████████████████████████████| 134 kB 44.4 MB/s 
     |████████████████████████████████| 1.1 MB 41.8 MB/s 
     |████████████████████████████████| 212 kB 46.0 MB/s 
     |████████████████████████████████| 127 kB 24.2 MB/s 
     |████████████████████████████████| 144 kB 46.2 MB/s 
     |████████████████████████████████| 271 kB 41.9 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

# Checkpoint we are using

In [2]:
checkpoint = "lighteternal/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext-finetuned-mnli"
# https://huggingface.co/lighteternal/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext-finetuned-mnli

# Change the goddamn label names **BEFORE YOU LOAD**
### Project
#### 0: Neutral/NOINFO
#### 1: Entailment/SUPPORT
#### 2: Contradiction/REFUTE

###BiomedNLP_PubMedBERT
#### 0: Entailment
#### 1: Neutral
#### 2: Contradiction

#Load train_update dataset

In [3]:
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, DataCollatorWithPadding

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# train_path = "/content/drive/MyDrive/train_update.csv"
train_path = "/content/drive/MyDrive/train_update_relabelled.csv"
raw_dataset = (load_dataset('csv', data_files=train_path))
train_dataset = (load_dataset('csv', data_files=train_path))
eval_dataset = (load_dataset('csv', data_files=train_path))
raw_dataset

Using custom data configuration default-77d6b9ff77906e78


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-77d6b9ff77906e78/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-77d6b9ff77906e78
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-77d6b9ff77906e78/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-77d6b9ff77906e78
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-77d6b9ff77906e78/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['ID', 'Claim', 'Evidence', 'Label'],
        num_rows: 1179
    })
})

In [6]:
print("Dataset columns originally looks like this:")
print(raw_dataset)

raw_dataset = raw_dataset.rename_column("ID", "idx")
raw_dataset = raw_dataset.rename_column("Claim", "sentence1")
raw_dataset = raw_dataset.rename_column("Evidence", "sentence2")
raw_dataset = raw_dataset.rename_column("Label", "label")

print("Dataset columns now looks like this:")
print(raw_dataset)

Dataset columns originally looks like this:
DatasetDict({
    train: Dataset({
        features: ['ID', 'Claim', 'Evidence', 'Label'],
        num_rows: 1179
    })
})
Dataset columns now looks like this:
DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence1', 'sentence2', 'label'],
        num_rows: 1179
    })
})


In [7]:
raw_dataset['train'] = raw_dataset.pop('train')

In [8]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence1', 'sentence2', 'label'],
        num_rows: 1179
    })
})

# Preparing for stratified kfold
### kfold is 10, each fold has 624 samples (208 per label)
### the leftover (555) is being used for evaluation

In [9]:
label0 = [] #Entailment
label1 = [] #Neutral
label2 = [] #Contradiction/REFUTE
for sequence in raw_dataset['train']:
  if sequence['label'] == 0:
    label0.append(sequence)
  elif sequence['label'] == 1:
    label1.append(sequence)
  elif sequence['label'] == 2:
    label2.append(sequence)
  
print('label0 length is: ' + str(len(label0)))
print('label1 length is: ' + str(len(label1)))
print('label2 length is: ' + str(len(label2)))

label0 length is: 602
label1 length is: 261
label2 length is: 316


# Preparing the kfold_train and kfold_eval

In [10]:
import random
import pandas as pd
from datasets import Dataset
for i in range(0,10):
  #this first part is for kfold_train

  label0_subset = random.sample(label0, 208)
  label1_subset = random.sample(label1, 208)
  label2_subset = random.sample(label2, 208)
  combined_subset = label0_subset + label1_subset + label2_subset
  random.shuffle(combined_subset)

  kfold_train = {
      "idx": [],
      "sentence1": [],
      "sentence2": [],
      "label": []
  }

  for sequence in combined_subset:
    kfold_train["idx"].append(sequence["idx"])
    kfold_train["sentence1"].append(sequence["sentence1"])
    kfold_train["sentence2"].append(sequence["sentence2"])
    kfold_train["label"].append(sequence["label"])

  kfold_train_dataset = Dataset.from_dict(kfold_train)
  name = "kfold_train_" + str(i)
  train_dataset[name] = kfold_train_dataset

  #this second part is for kfold_eval

  kfold_eval = {
      "idx": [],
      "sentence1": [],
      "sentence2": [],
      "label": []
  }

  for sequence in raw_dataset['train']:
    if sequence["idx"] not in kfold_train["idx"]:
      kfold_eval["idx"].append(sequence["idx"])
      kfold_eval["sentence1"].append(sequence["sentence1"])
      kfold_eval["sentence2"].append(sequence["sentence2"])
      kfold_eval["label"].append(sequence["label"])
  
  kfold_eval_dataset = Dataset.from_dict(kfold_eval)
  name = "kfold_eval_" + str(i)
  eval_dataset[name] = kfold_eval_dataset

In [11]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence1', 'sentence2', 'label'],
        num_rows: 1179
    })
})

In [12]:
train_dataset.pop('train')
train_dataset

DatasetDict({
    kfold_train_0: Dataset({
        features: ['idx', 'sentence1', 'sentence2', 'label'],
        num_rows: 624
    })
    kfold_train_1: Dataset({
        features: ['idx', 'sentence1', 'sentence2', 'label'],
        num_rows: 624
    })
    kfold_train_2: Dataset({
        features: ['idx', 'sentence1', 'sentence2', 'label'],
        num_rows: 624
    })
    kfold_train_3: Dataset({
        features: ['idx', 'sentence1', 'sentence2', 'label'],
        num_rows: 624
    })
    kfold_train_4: Dataset({
        features: ['idx', 'sentence1', 'sentence2', 'label'],
        num_rows: 624
    })
    kfold_train_5: Dataset({
        features: ['idx', 'sentence1', 'sentence2', 'label'],
        num_rows: 624
    })
    kfold_train_6: Dataset({
        features: ['idx', 'sentence1', 'sentence2', 'label'],
        num_rows: 624
    })
    kfold_train_7: Dataset({
        features: ['idx', 'sentence1', 'sentence2', 'label'],
        num_rows: 624
    })
    kfold_train_8: Dataset

In [13]:
eval_dataset.pop('train')
eval_dataset

DatasetDict({
    kfold_eval_0: Dataset({
        features: ['idx', 'sentence1', 'sentence2', 'label'],
        num_rows: 555
    })
    kfold_eval_1: Dataset({
        features: ['idx', 'sentence1', 'sentence2', 'label'],
        num_rows: 555
    })
    kfold_eval_2: Dataset({
        features: ['idx', 'sentence1', 'sentence2', 'label'],
        num_rows: 555
    })
    kfold_eval_3: Dataset({
        features: ['idx', 'sentence1', 'sentence2', 'label'],
        num_rows: 555
    })
    kfold_eval_4: Dataset({
        features: ['idx', 'sentence1', 'sentence2', 'label'],
        num_rows: 555
    })
    kfold_eval_5: Dataset({
        features: ['idx', 'sentence1', 'sentence2', 'label'],
        num_rows: 555
    })
    kfold_eval_6: Dataset({
        features: ['idx', 'sentence1', 'sentence2', 'label'],
        num_rows: 555
    })
    kfold_eval_7: Dataset({
        features: ['idx', 'sentence1', 'sentence2', 'label'],
        num_rows: 555
    })
    kfold_eval_8: Dataset({
      

# Load the test dataset

In [14]:
test_path = "/content/drive/MyDrive/test_phase_1_update_mod.csv"
test_dataset = load_dataset('csv', data_files=test_path)

Using custom data configuration default-2a2de451a581dbc4


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2a2de451a581dbc4/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
test_dataset = test_dataset.rename_column("ID", "idx")
test_dataset = test_dataset.rename_column("Claim", "sentence1")
test_dataset = test_dataset.rename_column("Evidence", "sentence2")
test_dataset = test_dataset.rename_column("Label", "label")
test_dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence1', 'sentence2', 'label'],
        num_rows: 150
    })
})

In [16]:
test_dataset['test'] = test_dataset.pop('train')
test_dataset

DatasetDict({
    test: Dataset({
        features: ['idx', 'sentence1', 'sentence2', 'label'],
        num_rows: 150
    })
})

#Get tokenizer, create tokenized dataset, create data collator (dynamic padding)

In [17]:
#get the tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

#create a tokenize function
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

#create the tokenized dataset
tokenized_train_dataset = train_dataset.map(tokenize_function)
tokenized_eval_dataset = eval_dataset.map(tokenize_function)
tokenized_test_dataset = test_dataset.map(tokenize_function)

#create the data_collator
data_collator = DataCollatorWithPadding(tokenizer)

Downloading:   0%|          | 0.00/388 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/221k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

  0%|          | 0/624 [00:00<?, ?ex/s]

...

  0%|          | 0/555 [00:00<?, ?ex/s]

  0%|          | 0/555 [00:00<?, ?ex/s]

  0%|          | 0/555 [00:00<?, ?ex/s]

  0%|          | 0/555 [00:00<?, ?ex/s]

  0%|          | 0/150 [00:00<?, ?ex/s]

# Defining model

In [18]:
from transformers import AutoModelForSequenceClassification

#get the model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/890 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

## Creating a Trainer and defining the arguments related to model

In [ ]:
from datasets import load_metric
import numpy as np

def compute_metrics(eval_preds):
    metric = load_metric("f1", "accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [20]:
from transformers import TrainingArguments, Trainer
from torch.utils.data import DataLoader

training_args = TrainingArguments("test-trainer",
                                  num_train_epochs=10,
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32,
                                  weight_decay=0.01
                                  )
                                  # evaluation_strategy="epoch")

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_dataset['kfold_train_0'],
    eval_dataset=tokenized_eval_dataset['kfold_eval_0'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics,
)

In [21]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 624
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 200


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=200, training_loss=0.167469539642334, metrics={'train_runtime': 260.8917, 'train_samples_per_second': 23.918, 'train_steps_per_second': 0.767, 'total_flos': 437146498966464.0, 'train_loss': 0.167469539642334, 'epoch': 10.0})

In [24]:
import numpy as np
predictions = trainer.predict(tokenized_eval_dataset['kfold_eval_0'])
preds = np.argmax(predictions.predictions, axis=-1)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 555
  Batch size = 32


In [26]:
from datasets import load_metric
metric = load_metric('glue', 'mnli')

In [27]:
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8666666666666667}

# 2

In [28]:
#2
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_dataset['kfold_train_1'],
    eval_dataset=tokenized_eval_dataset['kfold_eval_1'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 624
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 200


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=200, training_loss=0.03489361047744751, metrics={'train_runtime': 264.283, 'train_samples_per_second': 23.611, 'train_steps_per_second': 0.757, 'total_flos': 443921669472960.0, 'train_loss': 0.03489361047744751, 'epoch': 10.0})

# 3

In [32]:
#3
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_dataset['kfold_train_2'],
    eval_dataset=tokenized_eval_dataset['kfold_eval_2'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 624
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 200


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=200, training_loss=0.03332470417022705, metrics={'train_runtime': 261.6756, 'train_samples_per_second': 23.846, 'train_steps_per_second': 0.764, 'total_flos': 436151601355680.0, 'train_loss': 0.03332470417022705, 'epoch': 10.0})

In [29]:
predictions = trainer.predict(tokenized_eval_dataset['kfold_eval_1'])
preds = np.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds, references=predictions.label_ids)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 555
  Batch size = 32


{'accuracy': 0.9243243243243243}

# 8

In [50]:
#8
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_dataset['kfold_train_7'],
    eval_dataset=tokenized_eval_dataset['kfold_eval_7'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 624
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 200


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=200, training_loss=0.007779202461242676, metrics={'train_runtime': 256.4544, 'train_samples_per_second': 24.332, 'train_steps_per_second': 0.78, 'total_flos': 426367071133920.0, 'train_loss': 0.007779202461242676, 'epoch': 10.0})

In [47]:
predictions = trainer.predict(tokenized_eval_dataset['kfold_eval_6'])
preds = np.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds, references=predictions.label_ids)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 555
  Batch size = 32


{'accuracy': 0.9963963963963964}

In [51]:
model.save_pretrained("/content/drive/MyDrive/k-folds/8")

Configuration saved in /content/drive/MyDrive/k-folds/8/config.json
Model weights saved in /content/drive/MyDrive/k-folds/8/pytorch_model.bin


# Testing the phase1 data

In [52]:
predictions = trainer.predict(tokenized_test_dataset['test'])
preds = np.argmax(predictions.predictions, axis=-1)
print(len(preds))
for pred in preds:
  print(pred)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 150
  Batch size = 32


150
1
1
2
0
0
1
1
2
2
0
0
0
2
2
0
0
1
2
1
1
2
0
2
0
0
2
0
0
0
1
0
2
2
0
2
0
0
2
2
1
1
0
1
2
2
0
0
2
1
0
0
0
1
0
0
1
2
0
0
0
1
0
1
2
2
0
2
1
1
0
0
1
0
0
0
0
0
2
1
1
1
1
2
1
0
2
0
2
2
0
2
1
1
2
1
1
2
2
2
2
2
1
0
0
1
1
0
2
0
0
1
0
0
1
2
2
2
2
1
2
2
0
0
0
0
0
2
2
0
1
0
2
2
0
1
2
0
0
1
0
0
1
0
1
0
1
2
1
1
0
